In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import hdbscan
from sklearn.manifold import TSNE
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import plot_model

2023-04-03 22:14:43.774125: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/intel/compilers_and_libraries_2018.1.163/linux/tbb/lib/intel64_lin/gcc4.7:/opt/intel/compilers_and_libraries_2018.1.163/linux/compiler/lib/intel64_lin:/opt/intel/compilers_and_libraries_2018.1.163/linux/mkl/lib/intel64_lin::/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64/
2023-04-03 22:14:43.774186: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### DATA PREPARATION

In [6]:
# Read in the data
# Main change from v2 to v3 is that the specific regions have been replaced with general regions
cmd_data = pd.read_csv('/data2/home/prasannaiyer/Projects/SKU_Cluster_Local/SKU_Clusters/Data/cmd_attributes_v3_upload.csv',\
    encoding = 'latin-1', decimal = '.', thousands = ',')

In [7]:
attr_input = ['Attr1_Str_qu', 'Attr2_OpPr_qu','Attr3_Costamount_norm',
       'Attr4_RdDi_norm', 'Attr5_BrDi_norm', 'Attr5_Str_Type1',
       'Attr5_Str_Type2', 'Attr5_Str_Type3', 'Attr6_Major_Type1',
       'Attr6_Major_Type2', 'Attr9_Frctn0', 'Attr9_Frctn1', 'Attr10_Snsr0', 'Attr10_Snsr1']

In [8]:
attr_input_cost_model = ['Attr1_Str_qu', 'Attr2_OpPr_qu',
       'Attr4_RdDi_norm', 'Attr5_BrDi_norm', 'Attr5_Str_Type1',
       'Attr5_Str_Type2', 'Attr5_Str_Type3', 'Attr6_Major_Type1',
       'Attr6_Major_Type2', 'Attr9_Frctn0', 'Attr9_Frctn1', 'Attr10_Snsr0', 'Attr10_Snsr1']
attr_input_subset = ['Attr1_Str_qu', 'Attr2_OpPr_qu', 'Attr3_Costamount_norm',
       'Attr4_RdDi_norm', 'Attr5_BrDi_norm', 'Attr9_Frctn0', 'Attr9_Frctn1']
attr_input_subset_cost = ['Attr1_Str_qu', 'Attr2_OpPr_qu', 'Attr3_Costamount_norm',
       'Attr4_RdDi_norm', 'Attr5_BrDi_norm', 'Attr9_Frctn0', 'Attr9_Frctn1']

In [13]:
# create an autoencoder class. For init function, input would be the input data, output data, layers. Also in the init function, create the autoencoder model
# For the fit function, input would be the number of epochs, batch size, and the learning rate

class auto_encoder:
    def __init__(self, input_data, output_data, ae_layers, alpha_ae = 0.2):
        self.input_data = input_data
        self.output_data = output_data
        self.ae_layers = ae_layers
        self.alpha_ae = alpha_ae
        self.autoencoder = self.create_autoencoder()
                
    def create_autoencoder(self):
        # create the input layer
        encoder_input = Input(shape=(self.input_data.shape[1],))
        nn_layer = encoder_input
        # create the encoder layers
        for i in range(len(self.ae_layers)):
            layer_name = 'encoder_layer_' + str(i+1)
            layer_name = Dense(self.ae_layers[i])(nn_layer)
            layer_name = LeakyReLU(alpha=self.alpha_ae)(layer_name)
            nn_layer = layer_name
        encoder_output = nn_layer
        self.encoder_model = Model(encoder_input, encoder_output)
        # create the decoder layers
        for i in range(len(self.ae_layers)-2, -1, -1):
            layer_name = 'decoder_layer_' + str(i+1)
            layer_name = Dense(self.ae_layers[i])(nn_layer)
            layer_name = LeakyReLU(alpha=self.alpha_ae)(layer_name)
            nn_layer = layer_name
        decoder_output = nn_layer
        # create the output layer
        autoencoder_output = Dense(self.output_data.shape[1], activation = 'linear')(nn_layer)
        self.ae_model = Model(encoder_input, autoencoder_output)
        # compile the model
        self.ae_model.compile(optimizer='adam', loss='mse')

In [16]:
input_size = len(attr_input)
ae_layers = [input_size*2, 32, 16, 8]
ae_model = auto_encoder(cmd_data[attr_input], cmd_data[attr_input], ae_layers)

In [17]:
ae_model.ae_model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 14)]              0         
_________________________________________________________________
dense_17 (Dense)             (None, 28)                420       
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 28)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 32)                928       
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 32)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 16)                528       
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 16)                0   

In [28]:
# fit the model
ae_model_hist = ae_model.ae_model.fit(cmd_data[attr_input], cmd_data[attr_input], epochs = 500, batch_size = 32, verbose = 1)

Epoch 1/500
21/21 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 2/500
21/21 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 3/500
21/21 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 4/500
21/21 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 5/500
21/21 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 6/500
21/21 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 7/500
21/21 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 8/500
21/21 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 9/500
21/21 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 10/500
21/21 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 11/500
21/21 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 12/500
21/21 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 13/500
21/21 [=================

In [2]:
layer_list = [32,16,8]
for i in range(len(layer_list) - 1, -1, -1):
    print(layer_list[i])

8
16
32


In [4]:
for i in range(len(layer_list)):
    print(layer_list[i], "--", i)

32 -- 0
16 -- 1
8 -- 2


In [18]:
input_size = cmd_data[attr_input].shape[1]
l1_size = input_size*2
l2_size = 32
l3_size = 16
bottleneck_size = 8
encoder_input = Input(shape = (input_size, ))
# encoder layer 1
encoder_m5_l1 = Dense(l1_size)(encoder_input)
encoder_m5_l1 = LeakyReLU()(encoder_m5_l1)
# encoder layer 2
encoder_m5_l2 = Dense(l2_size)(encoder_m5_l1)
encoder_m5_l2 = LeakyReLU()(encoder_m5_l2)
# encoder layer 3
encoder_m5_l3 = Dense(l3_size)(encoder_m5_l2)
encoder_m5_l3 = LeakyReLU()(encoder_m5_l3)
# encoder bottleneck layer
encoder_m5_output = Dense(bottleneck_size)(encoder_m5_l3)
encoder_m5_output = LeakyReLU()(encoder_m5_output)
# decoder layer 1
decoder_m5_l1 = Dense(l3_size)(encoder_m5_output)
decoder_m5_l1 = LeakyReLU()(decoder_m5_l1)
# decoder layer 2
decoder_m5_l2 = Dense(l2_size)(decoder_m5_l1)
decoder_m5_l2 = LeakyReLU()(decoder_m5_l2)
# decoder layer 3
decoder_m5_l3 = Dense(l1_size)(decoder_m5_l2)
decoder_m5_l3 = LeakyReLU()(decoder_m5_l3)
# decoder output layer
decoder_m5_output = Dense(input_size, activation = 'linear')(decoder_m5_l3)
# define the autoencoder model
ae_fixed_m5 = Model(encoder_input, decoder_m5_output)

In [19]:
ae_fixed_m5.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 14)]              0         
_________________________________________________________________
dense_25 (Dense)             (None, 28)                420       
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 28)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 32)                928       
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 32)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 16)                528       
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 16)                0   

In [25]:
# compile the model
ae_fixed_m5.compile(optimizer='adam', loss='mse')
# fit the model
ae_fixed_m5_hist = ae_fixed_m5.fit(cmd_data[attr_input], cmd_data[attr_input], epochs = 500, batch_size = 32, verbose = 1)

Epoch 1/500
21/21 [==============================] - 1s 2ms/step - loss: 0.0031 
Epoch 2/500
21/21 [==============================] - 0s 2ms/step - loss: 6.8825e-04
Epoch 3/500
21/21 [==============================] - 0s 2ms/step - loss: 2.1397e-04
Epoch 4/500
21/21 [==============================] - 0s 2ms/step - loss: 1.1309e-04
Epoch 5/500
21/21 [==============================] - 0s 2ms/step - loss: 9.5213e-05
Epoch 6/500
21/21 [==============================] - 0s 2ms/step - loss: 8.9560e-05
Epoch 7/500
21/21 [==============================] - 0s 2ms/step - loss: 1.0692e-04
Epoch 8/500
21/21 [==============================] - 0s 2ms/step - loss: 9.5422e-05
Epoch 9/500
21/21 [==============================] - 0s 2ms/step - loss: 9.3104e-05
Epoch 10/500
21/21 [==============================] - 0s 2ms/step - loss: 8.9629e-05
Epoch 11/500
21/21 [==============================] - 0s 2ms/step - loss: 1.0414e-04
Epoch 12/500
21/21 [==============================] - 0s 2ms/step - loss: 1.1

In [27]:
# print the loss at epochs 100, 500, 999 and min loss. Format loss to 4 decimal places using f strings
print(f'Loss at epoch 100: {ae_fixed_m5_hist.history["loss"][99]:.4f}')
print(f'Loss at epoch 500: {ae_fixed_m5_hist.history["loss"][499]:.4f}')
#print(f'Loss at epoch 999: {ae_fixed_m5_hist.history["loss"][998]:.4f}')
print(f'Min loss: {min(ae_fixed_m5_hist.history["loss"]):.4f}')


Loss at epoch 100: 0.0002
Loss at epoch 500: 0.0001
Min loss: 0.0001


In [29]:
# print the loss at epochs 100, 500 and min loss for ae_model
print(f'Loss at epoch 100: {ae_model_hist.history["loss"][99]:.4f}')
print(f'Loss at epoch 500: {ae_model_hist.history["loss"][499]:.4f}')
print(f'Min loss: {min(ae_model_hist.history["loss"]):.4f}')

Loss at epoch 100: 0.0005
Loss at epoch 500: 0.0002
Min loss: 0.0001
